In [1]:
# ! pip install glob
import pyplume as pl
import netCDF4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob 

import xarray as xr 
import dask.array as da

from time import time
from datetime import datetime

In [2]:
# %git clone https://github.com/NCAR/wrf-python

In [3]:
dx=40; dy=40; dz=10

In [4]:
%cd /scratch/07528/ihd758/WRFout_h.2.5.restart/a.wrfout

/scratch/07528/ihd758/WRFout_h.2.5.restart/a.wrfout


In [5]:
# ds = xr.open_mfdataset('wrfout*',concat_dim="Time",combine='nested',parallel=True)
ds = xr.open_mfdataset('wrfout*',concat_dim="Time",combine='nested')

In [6]:
ds

<xarray.Dataset>
Dimensions:    (Time: 2400, bottom_top: 700, bottom_top_stag: 701, force_layers: 8, south_north: 100, south_north_stag: 101, west_east: 100, west_east_stag: 101)
Dimensions without coordinates: Time, bottom_top, bottom_top_stag, force_layers, south_north, south_north_stag, west_east, west_east_stag
Data variables: (12/26)
    Times      (Time) |S19 dask.array<chunksize=(300,), meta=np.ndarray>
    U          (Time, bottom_top, south_north, west_east_stag) float32 dask.array<chunksize=(300, 700, 100, 101), meta=np.ndarray>
    V          (Time, bottom_top, south_north_stag, west_east) float32 dask.array<chunksize=(300, 700, 101, 100), meta=np.ndarray>
    W          (Time, bottom_top_stag, south_north, west_east) float32 dask.array<chunksize=(300, 701, 100, 100), meta=np.ndarray>
    PH         (Time, bottom_top_stag, south_north, west_east) float32 dask.array<chunksize=(300, 701, 100, 100), meta=np.ndarray>
    PHB        (Time, bottom_top_stag, south_north, west_east) float32 dask.array<chunksize=(300, 701, 100, 100), meta=np.ndarray>
    ...         ...
    P_TOP      (Time) float32 dask.array<chunksize=(300,), meta=np.ndarray>
    TLP_STRAT  (Time) float32 dask.array<chunksize=(300,), meta=np.ndarray>
    UST        (Time, south_north, west_east) float32 dask.array<chunksize=(300, 100, 100), meta=np.ndarray>
    HFX        (Time, south_north, west_east) float32 dask.array<chunksize=(300, 100, 100), meta=np.ndarray>
    QFX        (Time, south_north, west_east) float32 dask.array<chunksize=(300, 100, 100), meta=np.ndarray>
    LH         (Time, south_north, west_east) float32 dask.array<chunksize=(300, 100, 100), meta=np.ndarray>
Attributes: (12/129)
    TITLE:                            OUTPUT FROM WRF V4.0.3 MODEL
    START_DATE:                      0001-01-01_00:00:00
    SIMULATION_START_DATE:           0001-01-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        101
    SOUTH-NORTH_GRID_DIMENSION:      101
    BOTTOM-TOP_GRID_DIMENSION:       701
    ...                              ...
    ISLAKE:                          0
    ISICE:                           0
    ISURBAN:                         0
    ISOILWATER:                      0
    HYBRID_OPT:                      0
    ETAC:                            0.0

In [12]:
def save_large_mat(data,filename):
    nx,ny,nz,nt = np.shape(data)
    t = int(nt/3)
    pl.savemat(data[:,:,:,:t],filename+"_1")
    pl.savemat(data[:,:,:,t:t*2],filename+"_2")
    pl.savemat(data[:,:,:,t*2:t*3],filename+"_3")

def trimdata(nxtrim, data):
    nt,nz,ny,nx = data.shape
    cx = round((nx-1)/2)
    cy = round((ny-1)/2)
    h = round(nxtrim/2)
    dataTrimed = data[:,:,(cy-h):(cy+h+1),(cx-h):(cx+h+1)]
    return dataTrimed

def loadCDF(D,dtype,time,trim = 45):

    D = D[dtype]
    if (dtype=="U"):
        D = (D[:,:,:,:-1] + D[:,:,:,1:])/2
    if (dtype=="W"):
        D = (D[:,:-1,:,:] + D[:,1:,:,:])/2 
    if (dtype=="V"):
        D = (D[:,:,:-1,:] + D[:,:,1:,:])/2

    D = trimdata(trim,D)

    D = np.array(D[:time,:,:,:])

    nt,nz,ny,nx = np.shape(D)
    temp = np.zeros((nx,ny,nz,nt))
    for t in range (nt):
        for i in range (nx):
            for j in range (nx):
                temp[i,j,:,t] = D[t,:,j,i]
    return temp


In [13]:
start_time = datetime.now()

T = loadCDF(ds,dtype="U",time=1800,trim = 45)
save_large_mat(T,"thermal_nbl_h.2.5_U")

T = loadCDF(ds,dtype="V",time=1800,trim = 45)
save_large_mat(T,"thermal_nbl_h.2.5_V")

T = loadCDF(ds,dtype="W",time=1800,trim = 45)
save_large_mat(T,"thermal_nbl_h.2.5_W")

end_time = datetime.now()
print("Time:", end_time - start_time)

Time: 0:34:40.186937


In [ ]:
# start_time = datetime.now()

# ts = ds["T"]
# T_flux = np.array(ts - ts.mean('Time'))

# # UU = ds["U"]
# # U_flux = np.array(UU - UU.mean('Time'))

# # VV = ds["V"]
# # V_flux = np.array(VV - VV.mean('Time'))

# # WW = ds["W"]
# # W_flux = np.array(WW - WW.mean('Time'))

# end_time = datetime.now()
# print("Time:", end_time - start_time)

In [ ]:
np.shape(T_test)
nt,nz,ny,nx = np.shape(T_test)
a = T_test[5,:,:,:]

In [ ]:
A = np.reshape(a,[nx,ny,nz],order ="C")

In [ ]:
plt.contour(temp[50,:,:,400].T)